In [1]:
# # A00_environment_and_genome_setup =============================================

# # in terminal, set up snm3C pipeline environment
# module load anaconda3 # or otherwise activate conda
# conda env create -f Documentation/snm3Cseq_taurus.yml # follow setup instructions

# # then, run the following commands
# qsub Scripts/A00a_genome_dl_index.sub  # ‡
# qsub Scripts/A00b_genome_prep_bismark.sub
# qsub Scripts/A00c_annotations_bed.sub # ‡

# # ‡ fast enough to run interactively

## minimal project directory setup

In [2]:
# # make sure that the project directory & minimal file structure exists
# # by running in terminal / bash
# dir_proj=/u/project/cluo_scratch/chliu/IGVF_iPSC_snm3Cseq_YZCL47
# mkdir $dir_proj; cd $dir_proj
# mkdir Metadata Notebooks Scripts

# # note that if running pipeline via .ipynb notebooks,
# # hitting "run all" on notebooks will fill up the Scripts folder

## snm3C_parameters (crucial to review!)

In [3]:
%%bash
cat > ../snm3C_parameters.env

# parameters file --------------------------------------------------------------
# note: filepaths are relative to project directory or absolute paths
# recommend using absolute filepaths for all "ref_dir"/genome related files by
# find & replace "/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda" --> 
#        folder where you want your genome ref_dir to reside

# primary analysis/project folder
dir_proj=/u/project/cluo_scratch/chliu/IGVF_iPSC_snm3Cseq_YZCL47

# folder with raw data
# (.fastq, for our group usually split across 4 lanes of a novaseq run)
dir_originalfastq=/u/project/cluo/Shared_Datasets/source_fastq/yzcl47

# scratch folder if available, otherwise can set be same as dir_proj
dir_scratch=/u/project/cluo_scratch/chliu

# reference genome/files
# these are generated after running scripts A00a-A00c
# alt: if a Hoffman2 user with access to our partition, can use other organisms in my Genomes dir
# (versions also exist +/- lambda spike-in : below is an IGVF hg38 + lambda)
ref_dir=/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda
ref_dir_bowtie1=/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda/bowtie1
ref_fasta=/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda/IGVF_GRCh38_plus_lambda.fa
ref_gtf=/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda/gencode.v43.chr_patch_hapl_scaff.annotation.gtf



# genome reference parameters  -------------------------------------------------
# point to $ref_dir directory, no need to change final file name
# (i.e., keep "chromsizes.tsv") besides organism
# again may be generated after running A00a-A00c

# reference genome
ref_chromsizes=/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda/chromsizes.tsv
ref_flat=/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda/annotations/refFlat.txt.gz 
ref_rrna=/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda/annotations/rRNA.intervallist 

# gene features
ref_genebody=/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda/annotations/genebody.bed
ref_geneslop2k=/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda/annotations/geneslop2k.bed
ref_exon=/u/project/cluo/chliu/Genomes/IGVF_hg38_pluslambda/annotations/exon.bed



# other miscellaneous parameters -----------------------------------------------
# OK as-is / defaults don't have to be changed

# metadata files
metadat_plate=Metadata/A01b_plate_metadata.csv
metadat_well=Metadata/A01c_well_filepath.csv

# fastqc (A02c, A03c)
wells_to_run=Scripts/A02c_random_fastqc_wells.txt
numwells_run=2
overwrite_random_wells="false"

## (A00a) download reference .fasta, index, get chrom sizes

In [4]:
# note: if using UCLA Hoffman2 cluster, can use my reference genomes
# just check that the "bowtie1" folder is present
# (see filepaths listed above)

# there may be extraneous files for genotyping pipelines that don't interfere with mCT or m3C
# (.fa.gz --> .bwt, .pac, .ann, .amb, .sa, .fai, .dict; generated with below commands)
# bwa index GRCh38.primary_assembly.genome.fa.gz
# gatk CreateSequenceDictionary -R GRCh38.primary_assembly.genome.fa

In [5]:
%%bash
cat > ../Scripts/A00a_genome_dl_index.sub

#!/bin/bash
#$ -cwd
#$ -o sublogs/A00a_genome_dl_index.$JOB_ID
#$ -j y
#$ -N A00a_genome_dl_index
#$ -l h_rt=8:00:00,h_data=8G



echo "Job $JOB_ID started on:   " `hostname -s`
echo "Job $JOB_ID started on:   " `date `
echo " "





# environment init -------------------------------------------------------------

. /u/local/Modules/default/init/modules.sh # <--
module load anaconda3 # <--
conda activate snm3Cseq_taurus # <--

export $(cat snm3C_parameters.env | grep -v '^#' | xargs) # <--



# download/extract ref ---------------------------------------------------------

cd ${ref_dir}

# double check filepaths and hard-coded names here # <--
# download IGVF consortium-specified .fa & .gtf as example (hard coded below)
# see https://lh3.github.io/2017/11/13/which-human-reference-genome-to-use
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.15_GRCh38/seqs_for_alignment_pipelines.ucsc_ids/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz # <--
wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_43/gencode.v43.chr_patch_hapl_scaff.annotation.gtf.gz # <--

# extract compressed files as needed
# put final, uncompressed names here into "snm3C_parameters.env" (as "ref_fasta" and "ref_gtf")
for gzfile in *gz
do
    gunzip -c ${gzfile} > ${gzfile/.gz/}
done

# note: if ref_fasta doesn't have .fa or .fasta file format
# bismark may have issues detecting it in ${ref_dir}
# rename .fna --> .fa, for example
for f in *fna; do mv ${f} ${f/fna/fa}; done



# account for Lambda spike-in --------------------------------------------------

# note on working with Lambda phage or other spike-in (bisulfite conversion efficiency QC)
# add to reference genome at this step. comment out below four lines--
# Escherichia phage Lambda, complete genome (GenBank: J02459.1) via NCBI

# wget "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file$=seqview&db=nuccore&report=fasta&id=215104" -O lambda.fa
# sed -i "1s/.*/>chrL/" lambda.fa
# cat GCA_000001405.15_GRCh38_no_alt_analysis_set.fa lambda.fa > IGVF_GRCh38_plus_lambda.fa 
# rm GCA_000001405.15_GRCh38_no_alt_analysis_set.fa



# index, chrom sizes -----------------------------------------------------------

# extract bp length/chromosome
samtools faidx ${ref_fasta}
cut -f 1-2 ${ref_fasta}.fai > chromsizes.tsv

# .fa --> .dict
picard CreateSequenceDictionary -R ${ref_fasta}





echo -e "\n\n'A00a_bwa_index' completed.\n\n"


echo "Job $JOB_ID ended on:   " `hostname -s`
echo "Job $JOB_ID ended on:   " `date `


## (A00b) prep bismark genome ref

In [6]:
%%bash
cat > ../Scripts/A00b_genome_prep_bismark.sub

#!/bin/bash
#$ -cwd
#$ -o sublogs/A00b_genome_prep_bismark.$JOB_ID
#$ -j y
#$ -N A00b_prep_bismark
#$ -l h_rt=24:00:00,h_data=8G
#$ -pe shared 4
#$ -hold_jid A00a_genome_dl_index



echo "Job $JOB_ID started on:   " `hostname -s`
echo "Job $JOB_ID started on:   " `date `
echo " "





# environment init -----------------------------------------------------------

. /u/local/Modules/default/init/modules.sh # <--
module load anaconda3 # <--
conda activate snm3Cseq_taurus # <--

export $(cat snm3C_parameters.env | grep -v '^#' | xargs) # <--



# bismark index -----------------------------------------------------------------

# if using TAURUS version of m3C pipeline ("snm3Cseq_taurus")
# use bowtie1 to index the genome

mkdir ${ref_dir}/bowtie1
cd ${ref_dir}/bowtie1

ln -s ${ref_fasta} input.fa

if [[ ! -s Bisulfite_Genome ]]
then
    bismark_genome_preparation $PWD --bowtie1
fi





echo -e "\n\n'A00b_genome_prep_bismark' completed.\n\n"


echo "Job $JOB_ID ended on:   " `hostname -s`
echo "Job $JOB_ID ended on:   " `date `


## (A00c) extract annotations from gtf

In [7]:
%%bash
cat > ../Scripts/A00c_gtf_annotations_bed.sub

#!/bin/bash
#$ -cwd
#$ -o sublogs/A00c_gtf_annotations_bed.$JOB_ID
#$ -j y
#$ -N A00c_gtf_annotations_bed
#$ -l h_rt=0:30:00,h_data=8G



echo "Job $JOB_ID started on:   " `hostname -s`
echo "Job $JOB_ID started on:   " `date `
echo " "






# environment init -----------------------------------------------------------

. /u/local/Modules/default/init/modules.sh # <--
module load anaconda3 # <--
conda activate snm3Cseq_taurus # <--

export $(cat snm3C_parameters.env | grep -v '^#' | xargs) # <--



# process .gtf --> .bed ------------------------------------------------------

mkdir ${ref_dir}/annotations

python Scripts/A00c_gtf_annotations_bed.py



# UCSC/Picard utilities for QC metrics ---------------------------------------

cd ${ref_dir}

# rRNA interval list
picard BedToIntervalList -I annotations/rRNA.bed -O \
    annotations/rRNA.intervallist -SD ${ref_fasta/fa/dict}

# ref flat
gtfToGenePred -genePredExt -geneNameAsName2 ${ref_gtf} refFlat.tmp.txt
cut -f 1-10 refFlat.tmp.txt > refFlat.tmp1
cut -f 12 refFlat.tmp.txt > refFlat.tmp2

paste refFlat.tmp1 refFlat.tmp2 > annotations/refFlat.txt
gzip annotations/refFlat.txt

rm refFlat.tmp*





echo -e "\n\n'A00c_gtf_annotations_bed' completed.\n\n"


echo "Job $JOB_ID ended on:   " `hostname -s`
echo "Job $JOB_ID ended on:   " `date `


In [8]:
%%bash
cat > ../Scripts/A00c_gtf_annotations_bed.py

# ==============================================================================
# A00c_gtf_annotations_bed.py 
# exports four annotation-related files to $ref_dir (reference genome)
# for mcds creation & down-stream analysis
# ==============================================================================

# recommend running interactively in python/Jupyter to check outputs
# works for .gtfs from GENCODE, but double check fmt for other sources

# # if running interactively, need to load some lines from snm3C_parameters.env
# # os.environ['ref_dir'] = "/u/project/cluo/chliu/Genomes/IGVF_hg38" or the below loop
# # (use absolute versus relative path of parameters.env file if below not working!)
# envvar_needed = ['dir_proj', 'ref_dir', 'ref_gtf', 'ref_chromsizes']
# try:
#     os.environ['ref_dir']
# except KeyError:
#     envspec = pd.read_csv("../snm3C_parameters.env", sep = "=", comment="#", header = None
#                ).set_axis(['varname', 'varpath'], axis = 1
#                ).query('varname in @envvar_needed')
#     for index, row in envspec.iterrows():
#         os.environ[row["varname"]] = row["varpath"]
# os.chdir(os.environ['dir_proj'])


# load packages ----------------------------------------------------------------

import pandas as pd
import os



# load reference info ----------------------------------------------------------

os.chdir(os.environ['ref_dir'])
os.makedirs("annotations/", exist_ok=True)

gtf_file = pd.read_csv(os.environ['ref_gtf'],
                       comment = "#", delimiter="\t", header = None)
chrom_sizes = pd.read_csv(os.environ['ref_chromsizes'], sep = "\t", header = None)
chrom_sizes.columns = ['#chr', 'chrlen'] 



# genebody ---------------------------------------------------------------------
# .gtf to .bed (1-based --> 0 based)
# columns: chr, start, end, ENSG identifier
# note that this contains mitochondrial, ribosomal, lncRNAs, etc;
# this may or may not be desireable in downstream analyses

genebody = gtf_file[gtf_file.iloc[:, 2] == 'gene'].iloc[:, [0, 3, 4, 8]]
genebody.iloc[:, 1] = genebody.iloc[:, 1] - 1 # start changes to 0-pos
genebody.columns = ['#chr', 'start', 'end', 'annot']
genebody.reset_index(inplace=True, drop=True)

# extract info from the annot column (;, ")
genebody['gene'] = genebody['annot'].transform(lambda x: str(x).split('\"')[1])
genebody['symbol'] = genebody['annot'].transform(lambda x: str(x).split('\"')[5])
genebody['type'] = genebody['annot'].transform(lambda x: str(x).split('\"')[3])

# .gtf checks: should be zero
if sum(genebody.gene.duplicated()) != 0 | sum(genebody.start >= genebody.end) != 0:
  print("WARNING: check .bed outputs; was gene info was processed correctly from .gtf?")

# export ENSG --> Symbol
genebody.drop('annot', axis = 1, inplace = True)
genebody.to_csv("annotations/ensembl_to_symbol.tsv", sep = "\t", index = False)

# bed4 format for .allcools
# (drops strand info, which can be useful for other software)
genebody = genebody.iloc[:, 0:4]
genebody.to_csv("annotations/genebody.bed", sep = "\t", index = False)



# gene +/- 2kb -----------------------------------------------------------------
# above, but padding a 2kb region
# past manuscripts do this for higher mC modality coverage, but less interpretable

g2k = genebody.copy()
g2k.iloc[:, 1] = g2k.iloc[:, 1] - 2000
g2k.iloc[:, 2] = g2k.iloc[:, 2] + 2000

# but check +/-2kb still within chromosome length
# (low # genes affected, but may cause downstream issues)
g2k.loc[g2k.start < 0, 'start'] = 0

g2k = pd.merge(g2k, chrom_sizes, on = '#chr') 
filter_chrlen = g2k.end > g2k.chrlen
g2k.loc[filter_chrlen, 'end'] = g2k.chrlen[filter_chrlen]
g2k.drop('chrlen', axis = 1, inplace=True)
g2k.to_csv("annotations/geneslop2k.bed", sep = "\t", index = False)



# exonic -----------------------------------------------------------------------
# exon-level annotations, useful for STAR exon-only quant, potential ASE

exon = gtf_file[gtf_file.iloc[:, 2] == 'exon'].iloc[:, [0, 3, 4, 8]]
exon.iloc[:, 1] = exon.iloc[:, 1] - 1 # start changes to 0-pos
exon.columns = ['#chr', 'start', 'end', 'annot']

exon['gene'] = exon['annot'].transform(lambda x: str(x).split('\"')[1])
exon['transcript'] =  exon['annot'].transform(lambda x: str(x).split('\"')[3])
exon = exon.drop('annot', axis = 1).reset_index(drop=True)
exon.to_csv("annotations/exon.bed", sep = "\t", index = False)



# rRNA genes -------------------------------------------------------------------
# for some QC metrics after RNA alignments

rRNA = genebody.loc[gtf_file.iloc[:, 8].str.contains("rRNA"), :]
rRNA.to_csv("annotations/rRNA.bed", sep = "\t", index = False)


